In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os
from datetime import datetime, timedelta

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Define data paths
data_dir = '../Data/'
application_file = os.path.join(data_dir, 'application_record.csv')
credit_file = os.path.join(data_dir, 'credit_record.csv')

# Load the data
application_df = pd.read_csv(application_file)
credit_df = pd.read_csv(credit_file)

# Display information about the datasets
print("Application Records Shape:", application_df.shape)
print("Credit Records Shape:", credit_df.shape)

# Preview application data
application_df.head()

Application Records Shape: (438557, 18)
Credit Records Shape: (1048575, 3)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
# Preview credit data
credit_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [4]:
# Check for missing values in application data
print("Missing values in application data:")
application_df.isnull().sum()

Missing values in application data:


ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [5]:
application_df.OCCUPATION_TYPE.replace(np.nan, 'Other', inplace = True)
application_df.OCCUPATION_TYPE.value_counts()

OCCUPATION_TYPE
Other                    134203
Laborers                  78240
Core staff                43007
Sales staff               41098
Managers                  35487
Drivers                   26090
High skill tech staff     17289
Accountants               15985
Medicine staff            13520
Cooking staff              8076
Security staff             7993
Cleaning staff             5845
Private service staff      3456
Low-skill Laborers         2140
Secretaries                2044
Waiters/barmen staff       1665
Realty agents              1041
HR staff                    774
IT staff                    604
Name: count, dtype: int64

In [6]:
# Check for missing values in credit data
print("Missing values in credit data:")
credit_df.isnull().sum()

Missing values in credit data:


ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [7]:
# Check for duplicates in application data
print(f"Number of duplicate rows in application data: {application_df.duplicated().sum()}")

# Check for duplicates in credit data
print(f"Number of duplicate rows in credit data: {credit_df.duplicated().sum()}")

Number of duplicate rows in application data: 0
Number of duplicate rows in credit data: 0


In [8]:
# Handle any duplicate rows if found
if application_df.duplicated().sum() > 0:
    application_df = application_df.drop_duplicates()
    
if credit_df.duplicated().sum() > 0:
    credit_df = credit_df.drop_duplicates()

In [9]:
# Convert DAYS_BIRTH to Age in Years
application_df['AGE_YEARS'] = round(abs(application_df['DAYS_BIRTH'] / 365.25), 1)

# Convert DAYS_EMPLOYED to Employment Duration in Years
# Note: Positive values indicate unemployment, so we need to handle this case
application_df['EMPLOYMENT_YEARS'] = application_df['DAYS_EMPLOYED'].apply(lambda x: 
                                                               round(abs(x / 365.25), 1) if x < 0 else 0)
application_df['IS_UNEMPLOYED'] = application_df['EMPLOYMENT_YEARS'] > 0

# Display the transformed columns
application_df[['DAYS_BIRTH', 'AGE_YEARS', 'DAYS_EMPLOYED', 'EMPLOYMENT_YEARS', 'IS_UNEMPLOYED']].head()

,DAYS_BIRTH,AGE_YEARS,DAYS_EMPLOYED,EMPLOYMENT_YEARS,IS_UNEMPLOYED
0,-12005,32.9,-4542,12.4,True
1,-12005,32.9,-4542,12.4,True
2,-21474,58.8,-1134,3.1,True
3,-19110,52.3,-3051,8.4,True
4,-19110,52.3,-3051,8.4,True


In [10]:
#Convert categorical variables to numerical variable
le = LabelEncoder()
for col in application_df.columns:
    if application_df[col].dtype == 'object':
        application_df[col] = le.fit_transform(application_df[col])
application_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE_YEARS,EMPLOYMENT_YEARS,IS_UNEMPLOYED
0,5008804,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,12,2.0,32.9,12.4,True
1,5008805,1,1,1,0,427500.0,4,1,0,4,-12005,-4542,1,1,0,0,12,2.0,32.9,12.4,True
2,5008806,1,1,1,0,112500.0,4,4,1,1,-21474,-1134,1,0,0,0,17,2.0,58.8,3.1,True
3,5008808,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,15,1.0,52.3,8.4,True
4,5008809,0,0,1,0,270000.0,0,4,3,1,-19110,-3051,1,0,1,1,15,1.0,52.3,8.4,True


In [15]:
# Define a function to classify customers as good or bad based on credit history
def create_target_variable(credit_data):
    # Group by ID and check if any STATUS is bad (0-5)
    credit_status = credit_data.copy()
    
    # Create a more balanced definition - consider even Status=1 as problematic
    credit_status['STATUS_NUM'] = credit_status['STATUS'].map(
        lambda x: int(x) if x in ['0', '1', '2', '3', '4', '5'] else -1)
    
    # Define target: 0 for bad credit (status 2 or worse), 1 for good credit
    worst_status = credit_status.groupby('ID')['STATUS_NUM'].max().reset_index()
    worst_status.columns = ['ID', 'WORST_STATUS']
    worst_status['TARGET'] = worst_status['WORST_STATUS'].apply(lambda x: 0 if x >= 2 else 1)
    
    # Get payment history length
    payment_history = credit_status.groupby('ID').size().reset_index()
    payment_history.columns = ['ID', 'MONTHS_HISTORY']
    
    # Merge worst status and payment history
    result = pd.merge(worst_status, payment_history, on='ID', how='inner')
    
    return result[['ID', 'TARGET', 'WORST_STATUS', 'MONTHS_HISTORY']]

# Create target variable
target_df = create_target_variable(credit_df)

# Preview target data
target_df.head()

,ID,TARGET,WORST_STATUS,MONTHS_HISTORY
0,5001711,1,0,4
1,5001712,1,0,19
2,5001713,1,-1,22
3,5001714,1,-1,15
4,5001715,1,-1,60


In [16]:
# Check the distribution of the target variable
print(target_df['TARGET'].value_counts())
print(f"Percentage of bad credit: {100-target_df['TARGET'].mean() * 100:.2f}%")

TARGET
1    45318
0      667
Name: count, dtype: int64
Percentage of bad credit: 1.45%


In [17]:
# Merge application data with target
final_df = pd.merge(application_df, target_df, on='ID', how='inner')

# Check the final dataset shape
print(f"Final dataset shape: {final_df.shape}")
print(f"Number of features: {final_df.shape[1] - 1}")
print(f"Number of samples: {final_df.shape[0]}")

Final dataset shape: (36457, 24)
Number of features: 23
Number of samples: 36457


In [18]:
# Create a processed data directory if it doesn't exist
processed_dir = '../Data/processed/'
os.makedirs(processed_dir, exist_ok=True)

# Save the processed data
final_df.to_csv(os.path.join(processed_dir, 'processed_data.csv'), index=False)

# Also save a version without one-hot encoding for EDA
eda_df = application_df[['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 
                        'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 
                        'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'AGE_YEARS', 
                        'EMPLOYMENT_YEARS', 'IS_UNEMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 
                        'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS']].copy()

eda_df = pd.merge(eda_df, target_df, on='ID', how='inner')
eda_df.to_csv(os.path.join(processed_dir, 'eda_data.csv'), index=False)

print("Processed data exported successfully.")

Processed data exported successfully.
